In [149]:
import tarfile
import json
from pathlib import Path
import pandas as pd
import re
import difflib

In [13]:
conda_info = !conda info --json
conda_info = json.loads("\n".join(conda_info))

In [22]:
build_root = Path("/home/weg/miniconda3/envs/_conda_forge/conda-bld/linux-64")

In [147]:
dfs = []
lfdfs = []
for tarball in list(build_root.glob("*offline*")):
    tf = tarfile.open(tarball)
    dfs += [pd.DataFrame([
        [
            tarball.name,
            Path(mbr.name).parent.name,
            Path(mbr.name).name,
            mbr.size,
        ]
        for mbr in tf.getmembers()
        if "tgz" in mbr.name
    ], columns=["package", "path", "name", "size"])]
    lfdfs += [pd.DataFrame([
        [
            tarball.name,
            Path(mbr.name).name,
            mbr.size,
            tf.extractfile(mbr).read().decode('utf-8')
        ]
        for mbr in tf.getmembers()
        if ".lock" in mbr.name
    ], columns=["package", "name", "size", "content"])]
df = pd.concat(dfs).sort_values("size")
lfdf = pd.concat(lfdfs).sort_values("size")
df.tail(10)

,package,path,name,size
798,jupyterlab-offline-0.31.12-0.tar.bz2,offline,@jupyterlab-services-1.1.4.tgz,552558
799,jupyterlab-offline-0.31.12-0.tar.bz2,offline,core-js-2.5.3.tgz,586699
1,jupyterlab-offline-bqplot-0.3.6-0.tar.bz2,extensions,bqplot-0.3.6.tgz,624035
800,jupyterlab-offline-0.31.12-0.tar.bz2,offline,handlebars-4.0.11.tgz,657458
801,jupyterlab-offline-0.31.12-0.tar.bz2,offline,codemirror-5.24.2.tgz,659243
802,jupyterlab-offline-0.31.12-0.tar.bz2,offline,font-awesome-4.7.0.tgz,671527
803,jupyterlab-offline-0.31.12-0.tar.bz2,offline,caniuse-db-1.0.30000815.tgz,1148279
804,jupyterlab-offline-0.31.12-0.tar.bz2,offline,fsevents-1.1.3.tgz,1837200
12,jupyterlab-offline-jupyter-widgets-jupyterlab-...,offline,jquery-ui-1.12.1.tgz,4053568
805,jupyterlab-offline-0.31.12-0.tar.bz2,offline,typescript-2.6.2.tgz,5572057


In [148]:
lfdf

,package,name,size,content
0,jupyterlab-offline-0.31.12-0.tar.bz2,jupyterlab-offline-0.31.12.lock,188763,# THIS IS AN AUTOGENERATED FILE. DO NOT EDIT T...
0,jupyterlab-offline-jupyter-widgets-jupyterlab-...,jupyterlab-offline-jupyter-widgets-jupyterlab-...,192916,# THIS IS AN AUTOGENERATED FILE. DO NOT EDIT T...
0,jupyterlab-offline-bqplot-0.3.6-0.tar.bz2,jupyterlab-offline-bqplot-0.3.6.lock,193427,# THIS IS AN AUTOGENERATED FILE. DO NOT EDIT T...


In [111]:
%%capture output
df.pivot_table(values="size", index=["package", "path"], 
               aggfunc=[len, sum])

In [117]:
# print(output.outputs[0].data["text/html"])

In [113]:
%%capture top_ten
df.tail(10)

In [118]:
# print(top_ten.outputs[0].data["text/html"])

In [141]:
df['upstream'] = df['name'].apply(lambda x: re.sub(r'-[\d\.]*.tgz', '', x))
dupes = df.pivot_table(values="size", index=["upstream"], aggfunc="count")
dupes[dupes["size"] > 1].sort_values("size")

,size
upstream,
acorn,2
hoek,2
http-signature,2
iconv-lite,2
inherits,2
is-accessor-descriptor,2
is-data-descriptor,2
is-descriptor,2
is-extendable,2


In [152]:
%%capture lockfiles
lfdf

In [157]:
print(lockfiles.outputs[0].data["text/html"])

<div>
<style scoped>
    .dataframe tbody tr th:only-of-type {
        vertical-align: middle;
    }

    .dataframe tbody tr th {
        vertical-align: top;
    }

    .dataframe thead th {
        text-align: right;
    }
</style>
<table border="1" class="dataframe">
  <thead>
    <tr style="text-align: right;">
      <th></th>
      <th>package</th>
      <th>name</th>
      <th>size</th>
      <th>content</th>
    </tr>
  </thead>
  <tbody>
    <tr>
      <th>0</th>
      <td>jupyterlab-offline-0.31.12-0.tar.bz2</td>
      <td>jupyterlab-offline-0.31.12.lock</td>
      <td>188763</td>
      <td># THIS IS AN AUTOGENERATED FILE. DO NOT EDIT T...</td>
    </tr>
    <tr>
      <th>0</th>
      <td>jupyterlab-offline-jupyter-widgets-jupyterlab-...</td>
      <td>jupyterlab-offline-jupyter-widgets-jupyterlab-...</td>
      <td>192916</td>
      <td># THIS IS AN AUTOGENERATED FILE. DO NOT EDIT T...</td>
    </tr>
    <tr>
      <th>0</th>
      <td>jupyterlab-offline-bqplot-0.3.6-0.tar.